In [32]:
import torch
torch.cuda.empty_cache()

In [33]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [34]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/configs/_custom_/swin_b.py')

root='../../../dataset/'

epoch = 'epoch_9'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 2

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = '/data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/work_dirs/swin_b_trash/'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.model.test_cfg.rpn.nms = dict(type='soft_nms', iou_threshold=0.3, min_score=0.05)
cfg.model.test_cfg.rcnn.nms = dict(type='soft_nms', iou_threshold=0.3, min_score=0.05)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [35]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=2,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [36]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/work_dirs/swin_b_trash/epoch_9.pth


In [37]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 7.7 task/s, elapsed: 632s, ETA:     0s

In [38]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.05778937 642.3632 252.52017 729.5588 338.1...,test/0000.jpg
1,0 0.40358865 768.9058 656.2394 879.10175 987.0...,test/0001.jpg
2,0 0.12428692 668.6103 221.77759 768.2425 337.8...,test/0002.jpg
3,0 0.15265234 2.8521423 525.00116 973.7456 971....,test/0003.jpg
4,0 0.3717195 173.26422 338.39944 891.36346 768....,test/0004.jpg
